# JOB SHOP SCHEDULING

In [2]:
import pandas as pd
import numpy as np
from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit
import pandas as pd 
import time
import collections
from ortools.linear_solver import pywraplp
import time

## Main functions

In [3]:
def extract_data(filename):
    # Initialize variables to store the data.
    nj = []
    nm = []
    times = []
    machines = []

    task_counter = 0
    task_matrix = []


    # Open the file and read each line.
    with open(filename, 'r') as file:
        contents = file.read()
        lines = contents.splitlines()

        for line in range(len(lines)):
            # Check if the line contains the number of jobs and machines.
            if 'Nb of jobs' in lines[line]:
                nj.append(lines[line + 1].split()[0])
                nm.append(lines[line + 1].split()[1])
                break
        
        num_jobs = int(nj[0])
        num_machines = int(nm[0])

        for line in range(len(lines)):

            if 'Nb of jobs' in lines[line]:
                continue

            elif ' ' in lines[line]:
                continue

            elif 'Times' in lines[line]:

                task_matrix.append(task_counter)
                task_matrix[task_counter] = []

                job_counter = 0

                pos = 0
                while pos < num_jobs:
                    pos += 1
                    time_var = lines[line + pos].split()
                    machine_var = lines[line + pos + num_jobs + 1].split()

                    task_matrix[task_counter].append([])
                    
                    #print('new times/machines pairs')
                    for aux in range(len(time_var)):
                        #print(f'{time_var[aux]} , {machine_var[aux]}')
                        task_matrix[task_counter][job_counter].append((int(time_var[aux]), int(machine_var[aux])))
                    
                    job_counter += 1

                task_counter +=1

        #print(task_matrix)

    return (num_jobs, num_machines, task_matrix)

In [4]:
num_jobs, num_machines, task_matrix = extract_data('tai15_15.txt')

In [5]:
a =[]
for i in task_matrix:
    for j in i:
        a.append(j)

In [6]:
def main(jobs_data):
    machines_count = num_machines
    all_machines = range(1,machines_count+1,1)
    # Computes horizon dynamically as the sum of all durations.
    horizon = sum(task[0] for job in jobs_data for task in job)

    # Named tuple to store information about created variables.
    task_type = collections.namedtuple('task_type', 'start end interval')
    # Named tuple to manipulate solution information.
    assigned_task_type = collections.namedtuple('assigned_task_type',
                                                'start job index duration')

    # Create the MIP solver with the CBC backend.
    solver = pywraplp.Solver('schedule_jobs_mip',
                            pywraplp.Solver.SCIP_MIXED_INTEGER_PROGRAMMING)

    # Creates job intervals and add to the corresponding machine lists.
    all_tasks = {}
    machine_to_intervals = collections.defaultdict(list)

    # Create binary variables for task overlap.
    x = {}
    M = 1000000  # Large constant for precedence constraints.
    for job_id, job in enumerate(jobs_data):
        for task_id, task in enumerate(job):
            for other_task_id, other_task in enumerate(job):
                if task_id != other_task_id:
                    x[job_id, task_id, other_task_id] = solver.BoolVar(
                        'x_%i_%i_%i' % (job_id, task_id, other_task_id))

    for job_id, job in enumerate(jobs_data):
        #print(job_id,job)
        for task_id, task in enumerate(job):
            #print(task_id+1, task)
            duration = task[0]
            machine = task[1]
            suffix = '_%i_%i' % (job_id, task_id)

            start_var = solver.IntVar(0, horizon, 'start' + suffix)
            #end_var = solver.IntVar(0, horizon, 'end' + suffix)
            end_var = start_var + duration
            #interval_var = solver.ComputeExactConditionNumber(0,horizon, name = 'interval' + suffix)
            all_tasks[job_id, task_id] = task_type(start=start_var,
                                                end = end_var,
                                                interval = None)
            machine_to_intervals[machine].append(start_var)
            #correto
    # # Create precedence constraints for tasks on the same machine.
    for job_id, job in enumerate(jobs_data):
        for task_id, task in enumerate(job):
            for other_task_id, other_task in enumerate(job):
                if task_id != other_task_id:
                    x[job_id, task_id, other_task_id] = solver.BoolVar(
                        'x_%i_%i_%i' % (job_id, task_id, other_task_id))
    # Add constraints to ensure that the start time, duration, and end time variables
    # represent a valid time interval.
    #solver.Add(end_var == start_var + interval_var)


    # Makespan objective.
    obj_var = solver.IntVar(0, horizon, 'makespan')
    for job_id, job in enumerate(jobs_data):
        solver.Add(obj_var >= all_tasks[job_id, len(job) - 1].end)
    solver.Minimize(obj_var)

    if solver.Solve() == pywraplp.Solver.OPTIMAL or solver.Solve() == pywraplp.Solver.FEASIBLE:
        print('Solution:')
        # Create one list of assigned tasks per machine.
        assigned_jobs = collections.defaultdict(list)
        for job_id, job in enumerate(jobs_data):
            for task_id, task in enumerate(job):
                machine = task[1]
                assigned_jobs[machine].append(
                    assigned_task_type(solver.Objective().Value(
                        ),
                                        job=job_id,
                                        index=task_id,
                                        duration=task[0]))
            # Create per machine output lines.
        output = ''
        for machine in all_machines:
            # Sort by starting time.
            assigned_jobs[machine].sort()
            sol_line_tasks = 'Machine ' + str(machine) + ': '
            sol_line = '           '

            for assigned_task in assigned_jobs[machine]:
                name = 'job_%i_task_%i' % (assigned_task.job,
                                            assigned_task.index)
                # Add spaces to output to align columns.
                sol_line_tasks += '%-15s' % name

                start = assigned_task.start
                duration = assigned_task.duration

                sol_tmp = '[%i,%i]' % (start,duration)
                # Add spaces to output to align columns.
                sol_line += '%-15s' % sol_tmp

            sol_line += '\n'
            sol_line_tasks += '\n'
            output += sol_line_tasks
            output += sol_line

        # Finally print the solution found.
        print(f'Optimal Schedule Length: {solver.Objective().Value()}')
        print(output)
    else:
        print('No solution found.')

In [7]:
list_files = ['tai15_15.txt','tai20_15.txt','tai20_20.txt','tai30_15.txt','tai30_20.txt','tai50_15.txt','tai50_20.txt','tai100_20.txt']
start_cell_time = time.time()
for i in range(len(list_files)):
    start_list_time = time.time()
    print(f'This is the file: {list_files[i]}')
    num_jobs, num_machines, task_matrix = extract_data(list_files[i])
    for i in range(len(task_matrix)):
        start_time = time.time()
        print (f'This is the list number: {i}')
        main(task_matrix[i])
        print("--- %.2f seconds ---" % (time.time() - start_time))
    print("--- %.2f seconds for the file---" % (time.time() - start_list_time))
print("--- %.2f seconds for the cell---" % (time.time() - start_cell_time))

This is the file: tai15_15.txt
This is the list number: 0
Solution:
Optimal Schedule Length: 97.0
Machine 1: job_0_task_13  job_1_task_10  job_2_task_8   job_3_task_5   job_4_task_13  job_5_task_11  job_6_task_12  job_7_task_2   job_8_task_4   job_9_task_5   job_10_task_3  job_11_task_2  job_12_task_7  job_13_task_14 job_14_task_9  
           [97,70]        [97,18]        [97,54]        [97,94]        [97,42]        [97,87]        [97,37]        [97,9]         [97,13]        [97,17]        [97,8]         [97,75]        [97,77]        [97,35]        [97,8]         
Machine 2: job_0_task_14  job_1_task_13  job_2_task_0   job_3_task_13  job_4_task_10  job_5_task_9   job_6_task_6   job_7_task_7   job_8_task_5   job_9_task_10  job_10_task_11 job_11_task_11 job_12_task_10 job_13_task_7  job_14_task_5  
           [97,83]        [97,99]        [97,4]         [97,92]        [97,93]        [97,26]        [97,19]        [97,62]        [97,87]        [97,69]        [97,93]        [97,45]        